In [1]:
%matplotlib inline

In [2]:
import os
from mxnet.gluon.model_zoo import vision
import numpy as np
from PIL import Image

from tvm import topi
import tvm
from tvm import te
from tvm import rpc, autotvm, relay
from tvm.contrib import graph_runtime, utils, download
from tvm.autotvm.measure.measure_methods import request_remote
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner

import vta
from vta.testing import simulator
from vta.top import graph_pack
import torch
import torchvision

{'TARGET': 'zcu104', 'HW_VER': '0.0.1'}
{'LOG_INP_WIDTH': 3, 'LOG_WGT_WIDTH': 3, 'LOG_ACC_WIDTH': 5, 'LOG_BATCH': 0, 'LOG_BLOCK_IN': 4, 'LOG_BLOCK_OUT': 4, 'LOG_UOP_WIDTH': 5, 'LOG_UOP_BUFF_SIZE': 15, 'LOG_INP_BUFF_SIZE': 15, 'LOG_WGT_BUFF_SIZE': 18, 'LOG_ACC_BUFF_SIZE': 17, 'LOG_INP_FACTOR_RESTRICT': 0, 'LOG_WGT_FACTOR_RESTRICT': 0, 'LOG_ACC_FACTOR_RESTRICT': 0, 'SHL_MEM_AXI_DATA_BITS': 64, 'SHL_VME_CLT_TAG_BITS': 19, 'DPI_LEN_BITS': 8, 'DPI_ADDR_BITS': 64, 'DPI_DATA_BITS': 64, 'DPI_TAG_BITS': 8, 'DPI_DELAY': 16, 'ENABLE_INSTRUCTION_CLP': 1, 'ENABLE_RUNTIME_GREEDY_UOP_LOADS': 1, 'ENABLE_RUNTIME_VALUE_BASED_UOP_CONSOLIDATION': 1, 'TARGET': 'zcu104', 'HW_VER': '0.0.1', 'ENABLE_VTHREAD': 1, 'VTHREAD_NAME': 'dthread'}


/home/srchand/anaconda3/envs/tvm-build-il-2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compile_network_pytorch(env, target, model, start_pack, start_pack_idx, stop_pack, stop_pack_idx):
    
    input_name = "input0"
    
    # Populate the shape and data type dictionary
    dtype_dict = {"data": "float32"}
    shape_dict = {"data": (env.BATCH, 3, 224, 224)}

    # Get off the shelf gluon model, and convert to relay
    pytorch_model = torch.hub.load('pytorch/vision', model, pretrained=True)
    input_shape = [1, 3, 224, 224]
    input_data = torch.randn(input_shape)
    scripted_model = torch.jit.trace(pytorch_model, input_data).eval()
    
    shape_list = [(input_name, input_shape)]
    mod , params = relay.frontend.from_pytorch(scripted_model, shape_list)    

    # Update shape and type dictionary
    shape_dict.update({k: v.shape for k, v in params.items()})
    dtype_dict.update({k: str(v.dtype) for k, v in params.items()})

    # Perform quantization in Relay
    # Note: We set opt_level to 3 in order to fold batch norm
    with tvm.transform.PassContext(opt_level=3):
        with relay.quantize.qconfig(global_scale=8.0, skip_conv_layers=[0]):
            mod = relay.quantize.quantize(mod, params=params)

    # Perform graph packing and constant folding for VTA target
    if target.device_name == "vta":
        assert env.BLOCK_IN == env.BLOCK_OUT
        relay_prog = graph_pack(
            mod["main"],
            env.BATCH,
            env.BLOCK_IN,
            env.BLOCK_OUT,
            env.WGT_WIDTH,
            start_name=start_pack,
            start_name_idx=start_pack_idx,
            stop_name=stop_pack,
            stop_name_idx=stop_pack_idx,
        )

    return relay_prog, params


def compile_network_mxnet(env, target, model, start_pack, start_pack_idx, stop_pack, stop_pack_idx):
        
    # Populate the shape and data type dictionary
    dtype_dict = {"data": "float32"}
    shape_dict = {"data": (env.BATCH, 3, 224, 224)}

    # Get off the shelf gluon model, and convert to relay
    gluon_model = vision.get_model(model, pretrained=True)
    mod, params = relay.frontend.from_mxnet(gluon_model, shape_dict)
    

    # Update shape and type dictionary
    shape_dict.update({k: v.shape for k, v in params.items()})
    dtype_dict.update({k: str(v.dtype) for k, v in params.items()})

    # Perform quantization in Relay
    # Note: We set opt_level to 3 in order to fold batch norm
    with tvm.transform.PassContext(opt_level=3):
        with relay.quantize.qconfig(global_scale=8.0, skip_conv_layers=[0]):
            mod = relay.quantize.quantize(mod, params=params)

    # Perform graph packing and constant folding for VTA target
    if target.device_name == "vta":
        assert env.BLOCK_IN == env.BLOCK_OUT
        relay_prog = graph_pack(
            mod["main"],
            env.BATCH,
            env.BLOCK_IN,
            env.BLOCK_OUT,
            env.WGT_WIDTH,
            start_name=start_pack,
            start_name_idx=start_pack_idx,
            stop_name=stop_pack,
            stop_name_idx=stop_pack_idx,
        )

    return relay_prog, params

In [ ]:
#input_name = "input0"
    
    # Populate the shape and data type dictionary
dtype_dict = {"data": "float32"}
shape_dict = {"data": (1, 3, 224, 224)}

# Get off the shelf gluon model, and convert to relay
gluon_model = vision.get_model("mobilenetv2_1.0", pretrained=True)
# pytorch_model = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)

input_shape = [1, 3, 224, 224]
# input_data = torch.randn(input_shape)
# scripted_model = torch.jit.trace(pytorch_model, input_data).eval()

# shape_list = [(input_name, input_shape)]
mod , params = relay.frontend.from_mxnet(gluon_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
        with relay.quantize.qconfig(global_scale=8.0, skip_conv_layers=[0]):
            mod = relay.quantize.quantize(mod, params=params)
            
print(mod.astext(show_meta_data=False))

## Start RPC Tracker
TVM uses an RPC session to communicate with Pynq boards.
During tuning, the tuner will send the generated code to the board and
measure the speed of code on the board.

To scale up tuning, TVM uses an RPC Tracker to manage multiple devices.
The RPC Tracker is a centralized controller node. We can register all devices to
the tracker. For example, if we have 10 Pynq boards, we can register all of them
to the tracker, and run 10 measurements in parallel, accelerating the tuning process.

To start an RPC tracker, run this command on the host machine. The tracker is
required during the whole tuning process, so we need to open a new terminal for
this command:

```bash
python -m tvm.exec.rpc_tracker --host=0.0.0.0 --port=9190
```
The expected output is:

```bash
INFO:RPCTracker:bind to 0.0.0.0:9190
```


## Register devices to RPC Tracker
Now we can register our devices to the tracker. The first step is to
build the TVM runtime for the Pynq devices.

Follow `vta-index`
to build the TVM runtime on the device. Then register the device to the tracker with:

```bash
python -m tvm.exec.rpc_server --tracker=[HOST_IP]:9190 --key=pynq
```
(replace :code:`[HOST_IP]` with the IP address of your host machine)

After registering devices, we can confirm it by querying the rpc_tracker:

```bash
python -m tvm.exec.query_rpc_tracker --host=0.0.0.0 --port=9190
```
For example, if we have 6 Pynq boards and 11 Raspberry Pi 3B,
the output can be

```bash
Queue Status
----------------------------------
key          total  free  pending
----------------------------------
pynq         6      6     0
rpi3b        11     11    0
----------------------------------
```
You can register multiple devices to the tracker to accelerate tuning.



## Set Tuning Options
Before tuning, we should apply some configurations.
Here we use an Pynq-Z1 board as an example.



In [4]:
# Tracker host and port can be set by your environment
tracker_host = os.environ.get("TVM_TRACKER_HOST", "127.0.0.1")
tracker_port = int(os.environ.get("TVM_TRACKER_PORT", 9190))

# Load VTA parameters from the 3rdparty/vta-hw/config/vta_config.json file
env = vta.get_env()

# This target is used for cross compilation. You can query it by :code:`gcc -v` on your device.
# Set ``device=arm_cpu`` to run inference on the CPU
# or ``device=vta`` to run inference on the FPGA.
device = "vta"
target = env.target if device == "vta" else env.target_vta_cpu

# Name of Gluon model to compile
# The ``start_pack`` and ``stop_pack`` labels indicate where
# to start and end the graph packing relay pass: in other words
# where to start and finish offloading to VTA.
network = "mobilenetv2_1.0"
start_pack = "cast"
start_pack_idx=7
stop_pack = "nn.global_avg_pool2d"
stop_pack_idx = 558

# Tuning option
log_file = "%s.%s.log" % (device, network)
tuning_option = {
    "log_filename": log_file,
    "tuner": "random",
    "n_trial": 1000,
    "early_stopping": None,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(),
        runner=autotvm.RPCRunner(
            env.TARGET,
            host=tracker_host,
            port=tracker_port,
            number=5,
            timeout=60,
#             module_loader=vta.module_loader(),
            # check_correctness=True, # TODO: re-enable when check_correctness works again.
        ),
    ),
}

In [5]:
# You can skip the implementation of this function for this tutorial.
def tune_tasks(
    tasks,
    measure_option,
    tuner="xgb",
    n_trial=1000,
    early_stopping=None,
    log_filename="tuning.log",
    use_transfer_learning=True,
):

    # create tmp log file
    tmp_log_file = log_filename + ".tmp"
    if os.path.exists(tmp_log_file):
        os.remove(tmp_log_file)

    for i, tsk in enumerate(reversed(tasks)):
        prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))

        # create tuner
        if tuner == "xgb" or tuner == "xgb-rank":
            tuner_obj = XGBTuner(tsk, loss_type="rank")
        elif tuner == "xgb_knob":
            tuner_obj = XGBTuner(tsk, loss_type="rank", feature_type="knob")
        elif tuner == "ga":
            tuner_obj = GATuner(tsk, pop_size=50)
        elif tuner == "random":
            tuner_obj = RandomTuner(tsk)
        elif tuner == "gridsearch":
            tuner_obj = GridSearchTuner(tsk)
        else:
            raise ValueError("Invalid tuner: " + tuner)

        if use_transfer_learning:
            if os.path.isfile(tmp_log_file):
                tuner_obj.load_history(autotvm.record.load_from_file(tmp_log_file))

        # do tuning
        tsk_trial = min(n_trial, len(tsk.config_space))
        tuner_obj.tune(
            n_trial=tsk_trial,
            early_stopping=early_stopping,
            measure_option=measure_option,
            callbacks=[
                autotvm.callback.progress_bar(tsk_trial, prefix=prefix),
                autotvm.callback.log_to_file(tmp_log_file),
            ],
        )

    # pick best records to a cache file
    autotvm.record.pick_best(tmp_log_file, log_filename)
    os.remove(tmp_log_file)

In [6]:
def register_vta_tuning_tasks():
    from tvm.autotvm.task import TaskExtractEnv

    @tvm.te.tag_scope(tag=topi.tag.ELEMWISE)
    def my_clip(x, a_min, a_max):
        """Unlike topi's current clip, put min and max into two stages."""
        const_min = tvm.tir.const(a_min, x.dtype)
        const_max = tvm.tir.const(a_max, x.dtype)
        x = te.compute(x.shape, lambda *i: tvm.te.min(x(*i), const_max), name="clipA")
        x = te.compute(x.shape, lambda *i: tvm.te.max(x(*i), const_min), name="clipB")
        return x

    # init autotvm env to register VTA operator
    TaskExtractEnv()

    @autotvm.template("conv2d_packed.vta")
    def _topi_nn_conv2d(*args, **kwargs):
        assert not kwargs, "Do not support kwargs in template function call"
        A, W = args[:2]

        with tvm.target.vta():
            res = vta.top.conv2d_packed(*args, **kwargs)
            res = topi.right_shift(res, 8)
            res = my_clip(res, 0, 127)
            res = topi.cast(res, "int8")

        if tvm.target.Target.current().device_name == "vta":
            s = vta.top.schedule_conv2d_packed([res])
        else:
            s = te.create_schedule([res.op])
        return s, [A, W, res]

In [ ]:
def tune_and_evaluate(tuning_opt):

    # Register VTA tuning tasks
    register_vta_tuning_tasks()

    # Perform task extraction on Relay program
    print("Extract tasks...")
    relay_prog, params = compile_network_mxnet(env, target, network, start_pack, start_pack_idx, stop_pack, stop_pack_idx)
    mod = tvm.IRModule.from_expr(relay_prog)
    tasks = autotvm.task.extract_from_program(
        mod,
        params=params,
        ops=(relay.op.get("nn.conv2d"),),
        target=target,
        target_host=env.target_host,
    )

    # filter out non-packed conv2d task
    tasks = list(filter(lambda t: len(t.args[0][1]) > 4 and "conv" in t.name, tasks))

    print(len(tasks))
    # We should have extracted 10 convolution tasks
    assert len(tasks) == 30
    print("Extracted {} conv2d tasks:".format(len(tasks)))
    for tsk in tasks:
        inp = tsk.args[0][1]
        wgt = tsk.args[1][1]
        batch = inp[0] * inp[4]
        in_filter = inp[1] * inp[5]
        out_filter = wgt[0] * wgt[4]
        height, width = inp[2], inp[3]
        hkernel, wkernel = wgt[2], wgt[3]
        hstride, wstride = tsk.args[2][0], tsk.args[2][1]
        hpad, wpad = tsk.args[3][0], tsk.args[3][1]
        print(
            "({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {})".format(
                batch,
                height,
                width,
                in_filter,
                out_filter,
                hkernel,
                wkernel,
                hpad,
                wpad,
                hstride,
                wstride,
            )
        )

    # We do not run the tuning in our webpage server since it takes too long.
    # Comment the following line to run it by yourself.
    #return

    # run tuning tasks
    print("Tuning...")
    tune_tasks(tasks, **tuning_opt)
    
    
    #device_host = os.environ.get("VTA_RPC_HOST", "192.168.2.99")
    #device_port = os.environ.get("VTA_RPC_PORT", "9091")
    
    # evaluate with tuning history
    if env.TARGET != "sim":
        # Get remote from fleet node
        remote = autotvm.measure.request_remote(
           env.TARGET, tracker_host, tracker_port, timeout=10000
        )
        #remote = rpc.connect(device_host, int(device_port))
        # Reconfigure the JIT runtime and FPGA.
        vta.reconfig_runtime(remote)
        vta.program_fpga(remote, bitstream=None)
    else:
        # In simulation mode, host the RPC server locally.
        remote = rpc.LocalSession()

    # compile kernels with history best records
    with autotvm.tophub.context(target, extra_files=[log_file]):
        # Compile network
        print("Compile...")
        if target.device_name != "vta":
            with tvm.transform.PassContext(opt_level=3, disabled_pass={"AlterOpLayout"}):
                lib = relay.build(
                    relay_prog, target=target, params=params, target_host=env.target_host
                )
        else:
            with vta.build_config(opt_level=3, disabled_pass={"AlterOpLayout"}):
                graph, lib, params = relay.build(
                    relay_prog, target=target, params=params, target_host=env.target_host
                )

        # Export library
        print("Upload...")
        temp = utils.tempdir()
        lib.export_library(temp.relpath("graphlib.tar"))
        remote.upload(temp.relpath("graphlib.tar"))
        lib = remote.load_module("graphlib.tar")

        # Generate the graph executor
        ctx = remote.ext_dev(0) if device == "vta" else remote.cpu(0)
#         m = graph_executo.GraphModule(lib["default"](ctx))
        m = graph_runtime.create(graph, lib, ctx)

        # upload parameters to device
        image = tvm.nd.array((np.random.uniform(size=(1, 3, 299, 299))).astype("float32"))
        m.set_input("data", image)

        # evaluate
        print("Evaluate inference time cost...")
        timer = m.module.time_evaluator("run", ctx, number=1, repeat=10)
        tcost = timer()
        prof_res = np.array(tcost.results) * 1000  # convert to millisecond
        print(
            "Mean inference time (std dev): %.2f ms (%.2f ms)"
            % (np.mean(prof_res), np.std(prof_res))
        )


# Run the tuning and evaluate the results
tune_and_evaluate(tuning_option)

Extract tasks...
30
Extracted 30 conv2d tasks:
(1, 7, 7, 320, 1280, 1, 1, 0, 0, 1, 1)
(1, 7, 7, 960, 320, 1, 1, 0, 0, 1, 1)
(1, 7, 7, 960, 960, 3, 3, 1, 1, 1, 1)
(1, 7, 7, 160, 960, 1, 1, 0, 0, 1, 1)
(1, 7, 7, 960, 160, 1, 1, 0, 0, 1, 1)
(1, 7, 7, 576, 160, 1, 1, 0, 0, 1, 1)
(1, 14, 14, 576, 576, 3, 3, 1, 1, 2, 2)
(1, 14, 14, 96, 576, 1, 1, 0, 0, 1, 1)
(1, 14, 14, 576, 96, 1, 1, 0, 0, 1, 1)
(1, 14, 14, 576, 576, 3, 3, 1, 1, 1, 1)
(1, 14, 14, 384, 96, 1, 1, 0, 0, 1, 1)
(1, 14, 14, 384, 384, 3, 3, 1, 1, 1, 1)
(1, 14, 14, 64, 384, 1, 1, 0, 0, 1, 1)
(1, 14, 14, 384, 64, 1, 1, 0, 0, 1, 1)
(1, 14, 14, 192, 64, 1, 1, 0, 0, 1, 1)
(1, 28, 28, 192, 192, 3, 3, 1, 1, 2, 2)
(1, 28, 28, 32, 192, 1, 1, 0, 0, 1, 1)
(1, 28, 28, 192, 32, 1, 1, 0, 0, 1, 1)
(1, 28, 28, 192, 192, 3, 3, 1, 1, 1, 1)
(1, 28, 28, 144, 32, 1, 1, 0, 0, 1, 1)
(1, 56, 56, 144, 144, 3, 3, 1, 1, 2, 2)
(1, 56, 56, 32, 144, 1, 1, 0, 0, 1, 1)
(1, 56, 56, 144, 32, 1, 1, 0, 0, 1, 1)
(1, 56, 56, 144, 144, 3, 3, 1, 1, 1, 1)
(1, 56, 56, 96,